In [ ]:
!pip install transformers
!pip install py2neo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Intent Recognition Model

In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TextClassificationPipeline

tokenizer = AutoTokenizer.from_pretrained("nlp-guild/bert-base-chinese-finetuned-intent_recognition-biomedical")
model = AutoModelForSequenceClassification.from_pretrained("nlp-guild/bert-base-chinese-finetuned-intent_recognition-biomedical")
ir_pipeline = TextClassificationPipeline(model = model, tokenizer = tokenizer)

label_set = [
'定义',
'病因',
'预防',
'临床表现(病症表现)',
'相关病症',
'治疗方法',
'所属科室',
'传染性',
'治愈率',
'禁忌',
'化验/体检方案',
'治疗时间',
'其他'
]


In [ ]:
def readable_results_ir(top_k:int, usr_query:str):
    raw = ir_pipeline(usr_query, top_k = top_k)
    def f(x):
        index = int(x['label'][6:])
        x['label'] = label_set[index]
    
    for i in raw:
        f(i)
    return raw
 
readable_results_ir(3,'得了心脏病怎么办')

[{'label': '治疗方法', 'score': 0.9994503855705261},
 {'label': '其他', 'score': 0.00018375989748165011},
 {'label': '临床表现(病症表现)', 'score': 0.00010841596667887643}]

# NER model

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("Adapting/bert-base-chinese-finetuned-NER-biomedical")
model = AutoModelForTokenClassification.from_pretrained("Adapting/bert-base-chinese-finetuned-NER-biomedical",revision='7f63e3d18b1dc3cc23041a89e77be21860704d2e')

from transformers import pipeline
ner_pipeline = pipeline('ner',model=model,tokenizer = tokenizer)

tag_set = [
 'B_手术',
 'I_疾病和诊断',
 'B_症状',
 'I_解剖部位',
 'I_药物',
 'B_影像检查',
 'B_药物',
 'B_疾病和诊断',
 'I_影像检查',
 'I_手术',
 'B_解剖部位',
 'O',
 'B_实验室检验',
 'I_症状',
 'I_实验室检验'
 ]
 
tag2id = lambda tag: tag_set.index(tag)
id2tag = lambda id: tag_set[id]

def readable_results_ner(query):
    result = ner_pipeline(query)

    results_in_word = {}
    j = 0
    while j < len(result):   
        i = result[j]
        entity = id2tag(int(i['entity'][i['entity'].index('_')+1:]))
        token = i['word']
        if entity.startswith('B'):
            entity_name = entity[entity.index('_')+1:]

            word = token
            j = j+1
            while j<len(result):
                next = result[j]
                next_ent = id2tag(int(next['entity'][next['entity'].index('_')+1:]))
                next_token = next['word']

                if next_ent.startswith('I') and next_ent[next_ent.index('_')+1:] == entity_name:
                    word += next_token
                    j += 1

                    if j >= len(result):
                        # results_in_word.append((entity_name,word))
                        if entity_name not in results_in_word.keys():
                            results_in_word[entity_name] = [word]
                        else:
                             results_in_word[entity_name].append(word)
                else:
                    # results_in_word.append((entity_name,word))
                    if entity_name not in results_in_word.keys():
                        results_in_word[entity_name] = [word]
                    else:
                        results_in_word[entity_name].append(word)
                    break

        else:
            j += 1
    
    return results_in_word


        

In [ ]:
print(readable_results_ner('淋球菌性尿道炎会引起头痛'))

{'疾病和诊断': ['淋球菌性尿道炎'], '症状': ['头痛']}


In [ ]:
readable_results_ir(3,'得了心肌炎怎么办')

[{'label': '治疗方法', 'score': 0.9994162321090698},
 {'label': '其他', 'score': 0.00015517184510827065},
 {'label': '临床表现(病症表现)', 'score': 0.0001521914527984336}]

# Conversation

In [ ]:
user_query = '得了心肌炎怎么办'

In [ ]:
user_intent = readable_results_ir(1,user_query)[0]['label']
user_intent

'治疗方法'

In [ ]:
disease = readable_results_ner(user_query).get('疾病和诊断',None)
disease

['心肌炎']

In [ ]:
class Template:
    def __init__(self):
        self.schema = {
    "定义":{
        "slot_list" : ["疾病和诊断"],
        "cql_template" : "MATCH(p:疾病) WHERE p.name= '<Disease>' RETURN p.desc",
        "reply_template" : "<Disease>是这样的：\n",
        "ask_template" : "您问的是<Disease>的定义吗？",
    },
    "病因":{
        "slot_list" : ["疾病和诊断"],
        "cql_template" : "MATCH(p:疾病) WHERE p.name= '<Disease>' RETURN p.cause",
        "reply_template" : "<Disease>疾病的原因是：\n",
        "ask_template" : "您问的是疾病<Disease>的原因吗？",
    },
    "预防":{
        "slot_list" : ["疾病和诊断"],
        "cql_template" : "MATCH(p:疾病) WHERE p.name= '<Disease>' RETURN p.prevent",
        "reply_template" : "关于<Disease>疾病您可以这样预防：\n",
        "ask_template" : "请问您问的是疾病<Disease>的预防措施吗？",
    },
    "临床表现(病症表现)":{
        "slot_list" : ["疾病和诊断"],
        "cql_template" : "MATCH(p:疾病)-[r:has_symptom]->(q:症状) WHERE p.name= '<Disease>' RETURN q.name",
        "reply_template" : "<Disease>疾病的病症表现一般是这样的：\n",
        "ask_template" : "您问的是疾病<Disease>的症状表现吗？",
    },
    "相关病症":{
        "slot_list" : ["疾病和诊断"],
        "cql_template" : "MATCH(p:疾病)-[r:acompany_with]->(q:疾病) WHERE p.name= '<Disease>' RETURN q.name",
        "reply_template" : "<Disease>疾病的具有以下并发疾病：\n",
        "ask_template" : "您问的是疾病<Disease>的并发疾病吗？",
    },
    "治疗方法":{
        "slot_list" : ["疾病和诊断"],
        "cql_template" : ["MATCH(p:疾病) WHERE p.name= '<Disease>' RETURN p.cure_way",
                        "MATCH(p:疾病)-[r:recommand_drug]->(q) WHERE p.name= '<Disease>' RETURN q.name",
                        "MATCH(p:疾病)-[r:recommand_recipes]->(q) WHERE p.name= '<Disease>' RETURN q.name"],
        "reply_template" : "<Disease>疾病的治疗方式、可用的药物、推荐菜肴有：\n",
        "ask_template" : "您问的是疾病<Disease>的治疗方法吗？",
    },
    "所属科室":{
        "slot_list" : ["疾病和诊断"],
        "cql_template" : "MATCH(p:疾病)-[r:cure_department]->(q:科室) WHERE p.name= '<Disease>' RETURN q.name",
        "reply_template" : "得了<Disease>可以挂这个科室哦：\n",
        "ask_template" : "您想问的是疾病<Disease>要挂什么科室吗？",
    },
    "传染性":{
        "slot_list" : ["疾病和诊断"],
        "cql_template" : "MATCH(p:疾病) WHERE p.name= '<Disease>' RETURN p.easy_get",
        "reply_template" : "<Disease>较为容易感染这些人群：\n",
        "ask_template" : "您想问的是疾病<Disease>会感染哪些人吗？",
    },
    "治愈率":{
        "slot_list" : ["疾病和诊断"],
        "cql_template" : "MATCH(p:疾病) WHERE p.name= '<Disease>' RETURN p.cured_prob",
        "reply_template" : "得了<Disease>的治愈率为：",
        "ask_template" : "您想问<Disease>的治愈率吗？",
    },
    "治疗时间":{
        "slot_list" : ["疾病和诊断"],
        "cql_template" : "MATCH(p:疾病) WHERE p.name= '<Disease>' RETURN p.cure_lasttime",
        "reply_template" : "疾病<Disease>的治疗周期为：",
        "ask_template" : "您想问<Disease>的治疗周期吗？",
    },
    "化验/体检方案":{
        "slot_list" : ["疾病和诊断"],
        "cql_template" : "MATCH(p:疾病)-[r:need_check]->(q:检查) WHERE p.name= '<Disease>' RETURN q.name",
        "reply_template" : "得了<Disease>需要做以下检查：\n",
        "ask_template" : "您是想问<Disease>要做什么检查吗？",
    },
    "禁忌":{
        "slot_list" : ["疾病和诊断"],
        "cql_template" : "MATCH(p:疾病)-[r:not_eat]->(q:食物) WHERE p.name= '<Disease>' RETURN q.name",
        "reply_template" : "得了<Disease>切记不要吃这些食物哦：\n",
        "ask_template" : "您是想问<Disease>不可以吃的食物是什么吗？",
    },
    "其他": None
}
        self.dunno_templates = [
            "人类的语言太难了！！",
            "没有理解您说的意思哦~",
            "非常抱歉，我还不知道如何回答您，我正在努力学习中~",
            "额~似乎有点不理解你说的是啥呢~~",
            "您说的我有点不明白，您可以换个问法问我哦~",
            "很抱歉没有理解你的意思呢~",
        ]

        self.greeting_templates = [
            "hi",
            "你好呀",
            "我是智能医疗诊断机器人，有什么可以帮助你吗",
            "hi，你好，你可以叫我小智",
            "你好，你可以问我一些关于疾病诊断的问题哦"
        ]

        self.goodbye_templates = [
            "再见，很高兴为您服务",
            "bye",
            "再见，感谢使用我的服务",
            "再见啦，祝你健康"
        ]

        self.self_intro_templates = [
            "我是小智，你的智能健康顾问",
            "你可以叫我小智哦~",
            "我是医疗诊断机器人小智"
        ]

In [ ]:
import random
from py2neo import Graph

class Conversation:
    def __init__(self) -> None:
        self.context = None # 正在讨论的疾病
        self.templates = Template()
        self.password = None # change password here
        self.kg = Graph("neo4j+s://f54cadff.databases.neo4j.io:7687", auth=("neo4j", self.password))
    
    def reply(self, user_query:str):
        if any(x in user_query for x in ['hello', 'hi','你好', '嗨嗨', '你好啊']):
            return random.choice(self.templates.greeting_templates)
        elif any(x in user_query for x in ['bye', 'goodbye','88', '再见', '谢谢', '感谢','拜拜','白白', '好的']):
            return random.choice(self.templates.goodbye_templates)
        elif any(x in user_query for x in ['名字', '是谁','who are you', '你是']):
            return random.choice(self.templates.self_intro_templates)

        user_intent = readable_results_ir(1,user_query)[0]['label']
        if user_intent == '其他':
            return random.choice(self.templates.dunno_templates)
        else:
            # get the slot list for this intent
            schema = self.templates.schema[user_intent]
            slot_list = schema['slot_list']

            ne =  readable_results_ner(user_query)
            slots = {}

            for slot in slot_list:
                slot_value = ne.get(slot,None)
                if slot_value is None:
                    if slot == '疾病和诊断' and self.context is not None:
                        slot_value = [self.context]
                        slots[slot] = slot_value
                    else:
                        return f'请您提供一下信息: {slot}.'
                else:
                    slots[slot] = slot_value
            

            # 到这里时该intent所有必要的slots已填充并保存于 $slots$
            disease = slots['疾病和诊断'][0]
            self.context = disease
            cql_templates = schema['cql_template']

            knowledge = ''

            if isinstance(cql_templates, list):
                for _ in cql_templates:
                    knowledge += self.query_kg(_.replace('<Disease>',disease)) +'\n'
            else:
                knowledge += self.query_kg(cql_templates.replace('<Disease>',disease))
            
            response = schema['reply_template'].replace('<Disease>',disease)
            response += knowledge

            # context management


            return response

    def query_kg(self,cql:str):
        try:   
            ret = ''
            data = self.kg.run(cql).data()
            for i in data:
                tmp = list(i.values())[0]
                if isinstance(tmp,list):
                    ret +=', '.join(tmp)
                else:
                    ret += tmp
                
                ret += ', '
    
            return ret[:-2]+'.'
        except Exception as e:
            print(e)
            return ''


In [ ]:
conv = Conversation()

In [ ]:
conv.reply('你干嘛')

'人类的语言太难了！！'

In [ ]:
print(conv.reply('得了心肌炎怎么办'))

心肌炎疾病的治疗方式、可用的药物、推荐菜肴有：
药物治疗, 支持性治疗.
转移因子胶囊, 滋心阴胶囊, 三磷酸腺苷二钠片, 辅酶Q10胶囊, 复方二氯醋酸二异丙胺片, 辅酶Q10片, 门冬氨酸钾镁片, 参龙宁心胶囊, 心欣舒胶囊, 软脉灵口服液, 屏风生脉胶囊.
小麦红枣猪脑汤, 莲子黑枣小麦汤, 雪菜牛肉粥, 草菇猪心肉片饭, 百合芝麻猪心汤, 牛肉汉堡包, 芹菜牛肉粥, 紫菜色拉卷.



In [ ]:
print(conv.reply('请问得了心脏病怎么办'))

心脏病疾病的治疗方式、可用的药物、推荐菜肴有：
手术治疗, 介入治疗, 药物治疗, 支持性治疗.
琥乙红霉素片, 依托红霉素颗粒, 琥乙红霉素颗粒, 依托红霉素片, 地高辛片.
素炒菠菜, 五香蒜苔, 素炒小白菜, 苦瓜粥, 银耳冬瓜羹, 芥菜炒蚕豆, 豌豆绿豆粥, 马兰头菜粥.



In [ ]:
print(conv.reply('心肌炎是什么'))

心肌炎是这样的：
心肌炎(myocarditis)是指由各种原因引起的心肌的局限性或弥漫性炎症，多种因素如感染、物理和化学因素均可引起心肌炎，所造成的心肌损害的轻重程度差别很大，临床表现各异，轻症患者无任何症状，而重症患者可发生心力衰竭、心源性休克甚至猝死。大部分患者经治疗可获得痊愈，有些患者在急性期之后发展为扩张型心肌病改变，可反复发生心力衰竭。虽然某些心肌炎由于在终期可过渡为充血性或限制性心肌病，而被某些学者视为继发性心肌病，但在发病学上心肌炎毕竟是可区分的疾病类型，引起心肌炎的原因很多：感染性因素，主要是病毒如柯萨奇病毒、艾柯病毒、流感病毒、腺病毒、肝炎病毒等;细菌如白喉杆菌、链球菌等;真菌;立克次体;螺旋体;原虫等。其中病毒性心肌炎最常见。自身免疫性疾病：如系统性红斑狼疮、巨细胞性心肌炎;物理因素：如胸部放射性治疗引起的心肌损伤;化学因素：如多种药物如一些抗菌素、肿瘤化疗药物等。.


In [ ]:
print(conv.reply('心肌炎要治疗多久'))

疾病心肌炎的治疗周期为：6个月.


In [ ]:
print(conv.context)

心肌炎


In [ ]:
conv.reply('他需要治疗多久')

'疾病心肌炎的治疗周期为：6个月.'

In [ ]:
conv.reply('他有没有并发症')

'心肌炎疾病的具有以下并发疾病：\n心源性休克.'

In [ ]:
usr = input()
while usr != 'exit':
    print(conv.reply(usr))
    print()
    usr = input()

你是谁
你可以叫我小智哦~

你知道得了心脏病怎么办吗
心脏病疾病的治疗方式、可用的药物、推荐菜肴有：
手术治疗, 介入治疗, 药物治疗, 支持性治疗.
琥乙红霉素片, 依托红霉素颗粒, 琥乙红霉素颗粒, 依托红霉素片, 地高辛片.
素炒菠菜, 五香蒜苔, 素炒小白菜, 苦瓜粥, 银耳冬瓜羹, 芥菜炒蚕豆, 豌豆绿豆粥, 马兰头菜粥.


要治疗多久啊
疾病心脏病的治疗周期为：1-3个月.

有没有禁忌啥的
得了心脏病切记不要吃这些食物哦：
白酒, 鹌鹑蛋, 鸡腿, 鸡肉.

有并发症吗
心脏病疾病的具有以下并发疾病：
脑梗塞.

好的 谢谢
再见，感谢使用我的服务

exit
